<a href="https://colab.research.google.com/github/KenYew/sre-notes/blob/main/networking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 Networking

---
## ❓ **What happens if I type example.com in my browser and hit enter?**

### Step 1: 📖 DNS
1. Checks if the DNS entry is available in any of the below cache layers.
  * Browser
  * Operating System
  * Local Area Network (Router)
  * DNS Server Cache (Internet Service Provider or Corporate Network)
2. If there is a DNS cache miss, the DNS resolver of the DNS server (from ISP or corporate network) performs a recursive DNS lookup.
3. DNS resolver recursively queries for the DNS answer from:
  * Root name server
  * Top level domain (TLD) name server
  * Route 53 DNS name server
4. DNS server returns the DNS answer (IP address of example.com) to the web browser and caches it. 
5. Web browser uses this resolved IP address to send requests to the web server of example.com
6. Web server of example.com returns the webpage to the web browser and displays the webpage.

### Step 2a: ⚡️ TCP

1. Packets from the client browser's request will be routed from the router, ISP and then to the Internet Exchange (switching between ISPs or networks) in TCP to find the server with the IP address to connect to.
1. When the browser finds the server on the Internet, it establishes a TCP connection with the server using a 3-way handshake.
1. If HTTPS is being used, a TLS handshake is performed to secure the communication.

#### 🤝 TCP 3-Way Handshake

| No. | Message | Description | 
|:--|:--|:--|
| 1. | SYN | SYN stands for synchronization.<li> Used to initiate and establish a connection. <li> Used to synchronize sequence numbers between devices.
| 2. | ACK	| ACK stands for acknowledgement. Helps to confirm to the other side that it has received the SYN.
| 3. | SYN-ACK	| SYN message from local device and ACK of the earlier packet.
| 4. | FIN |	FIN stands for Finished. Used to terminate a connection.
| 5. | Sequence Number | <li> A random 32 bit number which is assigned to the first bit of the data. <li> Generally, a sequence number is used only once in one connection. <li> For other data transmission in the same connection, some other random sequence number can be used.
| 6. | Acknowledgement Number | <li> It is the next sequence number that the acknowledgement sending device expects from the sender. <li> It is generally, 1 greater than the sequence number received from the sender.
| 7. | Window Size | It is the capacity up to which data can be received in the buffer.
| 8. | Maximum Segment Size | <li> It is the maximum acceptable size of each data segment by the connected device. <li> Above this size, the device will not be able to receive the data segments.

<img src="https://drive.google.com/uc?id=1s33vrvQDL4pFy4BflSBU8VwJecHo4ddB" align="middle" width="500px"/>

When establishing a connection using the 3-way handshake process in TCP:

1. `SYN`: The client establishes a connection with a server. It sends a segment with `SYN` and informs the server that the client wants to start communication, and with an initial sequence number.
    * For example, client begins the connection by sending the TCP `SYN` packet to server. The packets contain a random sequence number (e.g.: 4321) that indicates the beginning of sequence numbers that the client should transmit with the data.
2. `SYN-ACK`: The server then responds to the client's request with a `SYN-ACK` signal set. `ACK` helps signify the response of segment that is received and `SYN` signifies what sequence number it should start with the segments.
    * For example, after the server receives the `SYN` packet, and it then responds with `SYN ACK` that includes the server's own sequence number (e.g. 5501) and its acknowledgment number, that is the client's sequence number incremented by 1 (e.g. 4322).
3. `ACK`: Finally, the client acknowledges the response of the server, they both create a stable bi-directional connection and begin the actual data transfer process.
    * For example, client responds to server by sending the acknowledgment number that is usually the server’s sequence number incremented by 1 (e.g. 5502)

<img src="https://drive.google.com/uc?id=1xa7rC2vUdnQBPWK2x2Huk_evhCUO3GcF" align="middle" width="500px"/>

When terminating a connection using the 3-way handshake process in TCP:
1. `FIN`: The client sends the FIN to the server. When the client wants to terminate the connection, it sets the FIN flag as '1' and sends the message to the server with a random sequence number. 
2. `FIN ACK`: The server replies with the `FIN` and the `ACK` to the client. After receiving the client's termination request, the server sends an acknowledge to the client by setting the `ACK` flag to '1'. The acknowledgement number of the ACK is one more than the received sequence number. 
    * For example, if the client has sent the `FIN` with sequence number = 1000, then the server will send the ACK with acknowledgement number = 1001. 
    * Also, the server sets the `FIN` flag to '1' and sends it to the client, if the server also wants to terminate the connection. 
    * The sequence number used here for the FIN will be different from the client's FIN. 
    * After completion of this step, the connection is terminated from the client to the server-side.
3. `ACK`: The client sends the ACK to the server. After receiving the `FIN` from the server, the client sets the `ACK` flag to '1' and sends it with an acknowledgement number 1 greater than the server's `FIN` sequence number to the client. Here, the FIN flag is kept '0'. 
    * After completion of this step, the connection is now terminated from the server to the client-side.

#### 👮 TCP Congestion Control

* Sometimes, due to network congestion or flaky hardware connections, TLS packets will be dropped before they get to their final destination. 
* The sender then has to decide how to react. The algorithm for this is called TCP congestion control. 

    1. Client chooses a congestion window based on the maximum segment size (MSS) of the connection.
    1. For each packet acknowledged, the window doubles in size until it reaches the 'slow-start threshold'. In some implementations, this threshold is adaptive.
    1. After reaching the slow-start threshold, the window increases additively for each packet acknowledged. If a packet is dropped, the window reduces exponentially until another packet is acknowledged.

#### 🔐 TLS Handshake

<img src="https://drive.google.com/uc?id=1Qm3Te9rH-piAjkyWUQOh7J39QMzhQTl6" align="middle" width="500px"/>

1. The client computer sends a `ClientHello` message to the server with its Transport Layer Security (TLS) version, list of cipher algorithms and compression methods available.
1. The server replies with a `ServerHello` message to the client with the TLS version, selected cipher, selected compression methods and the server's public certificate signed by a CA (Certificate Authority). The certificate contains a public key that will be used by the client to encrypt the rest of the handshake until a symmetric key can be agreed upon.
1. The client verifies the server digital certificate against its list of trusted CAs. If trust can be established based on the CA, the client generates a string of pseudo-random bytes and encrypts this with the server's public key. These random bytes can be used to determine the symmetric key.
1. The server decrypts the random bytes using its private key and uses these bytes to generate its own copy of the symmetric master key.
1. The client sends a `Finished` message to the server, encrypting a hash of the transmission up to this point with the symmetric key.
1. The server generates its own hash, and then decrypts the client-sent hash to verify that it matches. If it does, it sends its own `Finished` message to the client, also encrypted with the symmetric key.
1. From now on the TLS session transmits the application (HTTP) data encrypted with the agreed symmetric key.

### Step 2b: 🌍 CDN

1. The CDN caches static and dynamic content closer to the browser using edge locations. 
2. It is a global distributed network of caching servers to improve performance of a website by bringing content closer to your users. 
3. Instead of relying on the public internet routing infrastructure (subject to extra hops, redeliveries and packet loss), client browser request gets to ride on the AWS Global Network. 
4. Request is intelligently routed through the most performant location to deliver content to your browser.

### Step 3: 💻 HTTP

1. When the browser has an established TCP connection to the server, it follows the rules of communication of HTTP. 
2. Browser sends an HTTP request to the server to request contents of a page. 
3. HTTP request consists of request line, headers (metadata of the request) and body. 
    * Request line contains the following:
        * a request method, which is one of `GET, POST, PUT, PATCH, DELETE`, or a handful of other HTTP verbs
        * the path, pointing to the requested resources
        * the HTTP version to communicate with

    * Request headers pass extra information along from client that help route the request and indicate what type of client is making (user agent). 
    ```
    Host: jennapederson.dev
    User-Agent: curl/7.64.1
    Accept: */*
    ```
    * Body is usually empty for `GET` but for `POST, PUT or PATCH`, the body will contain data from the client to create or update. It can be of different formats and the server understands the format based on the request header, `Content-Type`. 
4. Once the server has received the request from the client, the server processes it and sends back a response. 
5. Response 200 Status code is received by the browser and renders the response. 
6. Browser inspects the response headers and renders the response.
    * Example: If `Content-Type` is HTML, browser knows how to render the resource in HTML. It may make additional requests to get JavaScript, CSS, images and data.

---
## 🔐 Public Key Infrastructure

#### **Symmetric Encryption**
1. In symmetric encryption, the same private key is used for both the sender and receiver to decrypt the data. 
2. Hence, any hacker sniffing the communication channel can have access to both the private key and data to decrypt. 

#### **Asymmetric Encryption**
1. In asymmetric encryption, a pair of keys are used instead; private and public keys. 
2. Public keys are sent and used by both the sender and receiver but they can only be used for encrypting data. 

#### **SSH Example**
1. SSH connection is an example of an asymmetric encryption. 
2. To access a remote server securely, the user runs ssh-keygen to generate a key pair (`id_rsa` - private key and `id_rsa.pub` - public key). 
3. In the remote server, add the public key entry into the `~/.ssh/authorized_keys` file. 
4. The user runs `ssh -i id_rsa user1@server1` to securely access the remote server using their private key. 
5. Multiple users can generate their own key pairs and add their public keys into the remote servers .ssh directory too for access. 

#### **Web Server Example**
The idea is to transfer the user's private symmetric keys to the web server securely using the web server's public key via asymmetric encryption. This will ensure only the user and web server have access to the user's private symmetric keys for HTTPs communication.
1. The web server generates a key pair using:
```python
> openssl genrsa -out my-webserver.key 1024
my-webserver.key # public key generated
> openssl rsa -in my-webserver.key -pubout > my-webserver.pem 
my-webserver.key my-webserver.pem # key pairs generated 
```
The web server keeps the private key and sends out the public key.
1. The user's browser receives the public key and uses it to encrypt their private key. 
1. The user's browser sends the encrypted user private key to the web server.
1. Since the encrypted user private key is encrypted using the web server's public key, the web server can decrypt it using the web server's private key.
1. Both parties have the same symmetric private keys that they can use to encrypt and decrypt data securely. 
1. Hackers that intercept the communication channel only has access to the web server's public key which can only encypt but not decrypt data. 

#### **Hacker's Spoof Web Server**
1. The same asymmetric cryptography can be used to create an encrypted symmetric communication channel between the user and the hacker's spoof web server. 
1. The hacker can reroute your traffic to the hacker's web server and send you their public key. 
1. We need to find a way to verify the public keys we receive from any web server.

#### **Web Server with Certificates Example**
The web server will send you a certificate with the public key instead. The certificate will contain:
    - Issuer Name (the name of the certificate authority)
    - Serial Number of the Data
    - Signature Algorithm (e.g.: SHA-256)
    - Validatity Period/Date
    - Subject Name (the main subject name of the web server)
    - Subject Alternative Name (the alternative/aliases of the web server)
    - Subject Public Key Info (the value of the public key)

#### **Hacker's Spoof Web Server with Certificates**
1. The hacker could do the same and send you a certificate with the public key too containing spoof certificate information. 
1. Hence, the most important factor to determine if the certificate is legitimate is which certificate authority signed the certificate?
1. An unsafe certificate will most likely be a self-signed certificate.  
1. The web browser checks the certificate authority (CA) who signed the public certificate received by the web server and may warn you if the CA is not trusted.

#### **Certificate Authorities**
1. CAs are well-known, trusted organizations (e.g.: Symantec, digitcert, GlobalSign) that can sign and validate your web server's certificates.
1. When the web server generates a key pair using:
```python
> openssl genrsa -out my-webserver.key 1024
my-webserver.key # public key generated
> openssl rsa -in my-webserver.key -pubout > my-webserver.pem 
my-webserver.key my-webserver.pem # key pairs generated 
```
The web server can then also perform a Certificate Signing Request (CSR) using: 
```python 
> openssl req -new -key my-webserver.key -out my-webserver.csr -subj "/C=US/ST=CA/O=MyOrg, Inc./CN=my-myserver.com"
my-webserver.key my-webserver.csr
```
1. The CA receives your CSR and validate your information. If verification is successful, they will sign and send back your certificate. 
1. The web server will now be able to use and send out public certificates that are signed by a Trusted CA that are trusted by the web browser.
1. If a hacker were to make a CSR to the CA, the CA will reject the hacker's CSR if the hacker's verification fails. 

#### **How can we trust Public Certificate Authorities?**
1. The CA generates their own private and public keys.
1. The private keys are used to sign certificates by the CAs for every CSR.
1. The public keys of all CAs are built in to every web browser's local database (this can viewed in the `Trusted Root Certificate Authorities` tab in the browser's certificate setting).
1. The browser uses the CA's public key to validate any incoming public certificates that was signed by the CA.

#### **How can we trust Private Certificate Authorities?**
1. Internal websites such as your company's payroll or intranet domains cannot use the public CAs to validate their private certificates.
1. Hence, companies host their own private CAs. 
1. Private CA servers are deployed internally to sign public certificates using their private key and distribute signed, trusted public certificates within the internal network. 
1. The public keys of your internal private CA servers can then be installed in all of your employee's web browser.
1. This will help establish secure connectivity within your organization. 

#### **Summary**
1. Data is encrypted using asymmetric encyption using a pair of private and public keys. 
1. A system administrator uses asymmetric encryption using key pairs to secure connectivity to the web servers. 
1. The web servers uses a pair of private and public keys to establish secure HTTPs traffic. 
1. The web server first sends a Certificate Signing Request (CSR)  to get their public key (in a form of a public certificate) signed by a Trusted Certificate Authority (CA). 
1. The CA uses its private key to sign the public certificate in the CSR if validation is successful.
1. All user's web browsers would already have a copy of the CA's public key in the local database.
1. The signed public certificate is sent back to the web server (who performed the CSR) and is configured with that certificate. 
1. Whenever a user accesses the web server, the signed public certificate is sent to the user's web browser.
1. The user's web browser uses the CA's public key to validate the incoming signed public certificate from the web server. 
1. The web browser then generates its own symmetric private key to establish a secure HTTPs channel with the web server.
1. The web browser uses the verified and signed public certificate from the web server to encrypt its symmetric private key. 
1. The encrypted symmetric private key is sent to the web server.
1. Since the encypted user private key was encrypted with the server's public key, it can use its private key to decrypt the data and retrieve the user's symmetric private key. 
1. Both the user and server now have copies of the user's private key that have been sent over securely and will be used for secure HTTPs communications between both parties moving forward. 
1. Once the user establishes trust with the website, the user can use their username and password to authenticate to the web server. 

#### **How can web servers trust incoming clients?**
1. Web servers can in turn request a certificate from the client. 
1. The client generates a pair of keys and have their public certificate (with the public key within) signed by a trusted CA via a CSR. 
1. The CA verifies the identity of the client and if the validation phase passes, the CA signs the certificate. 
1. The client then sends the signed public certificate to the web server 

| No. | Certificate Type | Certificate Extensions | Examples | 
|:--|:--|:--|:--|
| 1. | Public Key | `*.crt`, `*.pem` | 1. server.crt <br/> 2. server.pem <br/> 3. client.crt <br/> 4. client.pem |
| 2. | Private Key | `*.key`, `*-key.pem` | 1. server.key <br/> 2. server-key.pem <br/> 3. client.key <br/> 4. client-key.pem |









---
## 🌥 Secure Shell Host (SSH)

### **📖 SSH Glossary**
| No. | Name | Description |
|:--|:--|:--|
| 1. | sshd | This is the ssh daemon that runs on the server. |
| 2. | ssh | The ssh client used to connect to the SSH server from a remote system. |
| 3. | scp | This utility securely copies files between systems. |
| 4. | sftp | This utility securely copies files between systems, acting like FTP. |
| 5. | slogin | Like SSH, this utility is used to access the shell prompt remotely. |
| 6. | ssh-keygen | Used to create user's public/private keys. Can also create self-signed certificates. |

### **⚙ Configuration**
1. To configure the **sshd server**, edit the `/etc/ssh/sshd_config` file.
    * Make sure to run `systemctl daemon-reload && systemctl restart sshd` to update new configuration changes in the service.
    * Below are examples of useful `sshd` configuration parameters:

| No. | Option | Description |
|:--|:--|:--|
| 1. | AllowUsers | Restricts logins to the SSH server to only the users listed. Specify a list of users separated by spaces.
| 2. | DenyUsers | Prevents users listed from logging in through the SSH server. Specify a list of users separated by spaces.
| 3. | PermitRootLogin | Specifies whether root can authenticate through the SSH server. Set the value to `no` to disallow users from logging in as root. 

2. To configure the **ssh client**, edit the `/etc/ssh/ssh_config` or `~/.ssh/config` file.

### **💻 SSH Commands**
1. Username login: `ssh -l <user_name> <ip_address_of_remote_server>` 
1. Keypair login: `ssh -i myKeyPair.pem user@ec2-public-dns-name` 
1. File transfer to remote: `scp -i myKeyPair.pem /path/myFile.txt user@ec2-public-dns-name:path/`
1. File transfer from remote: `scp -i myKeyPair.pem user@ec2-public-dns-name:path/myFile.txt /path/myFile.txt`
1. SFTP to remote: `sftp <user_name>@<ip_address_of_remote_server>`

### **🔐 SSH Example with Password**
1. In asymmetric encryption, the SSH server sends its public keys to the SSH client.
1. The SSH server stores its keys in the following files: 
    * **Private key**: `/etc/ssh/ssh_host_key`
    * **Public key**: `/etc/ssh/ssh_host_key.pub`
1. The client receives the public key from the SSH server and checks if it already has a copy of that key. 
1. The client stores public keys from remote systems in:
    * `/etc/ssh_known_hosts`
    * `~/.ssh/known_hosts`
1. If the client trusts the server, it generates a symmetric key and uses the server's public key to encrypt it. 
1. The client sends the encrypted symmetric key to the server and the server decrypts it using its own private key.
1. Both systems now have the same symmetric key for **faster symmetric encryption** during the SSH session.
1. The client can now securely authenticate with their username and password because everything communicated will be encrypted.

### **🔓 SSH Example without Password**
1. The client must first generate their own key pair using `ssh-keygen -t rsa` or `ssh-keygen -t dsa` (both encryption methods required for SSH 2.0). These key pairs will be saved as:
    * `~/.ssh/id_rsa`
    * `~/.ssh/id_rsa.pub`
    * `~/.ssh/id_dsa`
    * `~/.ssh/id_dsa.pub`
1. The client needs to securely copy their public key and add it to the `~/.ssh/authorized_keys/` file of the remote server.
1. Establish a standard SSH session + password with the remote server (e.g.: `ssh -l <user_name> <ip_address_of_remote_server>`) and then run `scp ~/.ssh/id_rsa.pub <user_name>@<ip_address_of_remote_server>:myKeyFile` or `cat myKeyFile >> ~/.ssh/authorized_keys`. This will enable **public key authentication**. 
1. Finally, configure the `ssh-agent` to eliminate the need to enter a passphrase every time an SSH conection is established. 
1. The below `ssh-agent` command caches the keys once added to the agent with `ssh-add`: 
```bash
> ssh-agent bash
> ssh-add ~/.ssh/id_rsa
> ssh-add ~/.ssh/id_dsa
> Enter the passphrase of the key file: 
```
1. Once done, the `ssh-agent` stores the passphrase in memory and listens for SSH requests. It automatically provides the key passphrase when requested.
1. Running `ssh -l <user_name> <ip_address_of_remote_server>` no longer prompts for any private key passphrase. 
1. The client can also use their private key to SSH to the remote server using `ssh -i id_rsa user@server`
